In [1]:
#| default_exp methods.vae_cf

In [2]:
from ipynb_path import *

In [3]:
#| export
from cfnet.import_essentials import *
from cfnet.nets import MLP
from cfnet.training_module import BaseTrainingModule, BaseCFExplanationModule
from cfnet.utils import validate_configs, sigmoid, grad_update, cat_normalize, make_model, init_net_opt
from cfnet.utils import accuracy, proximity
from cfnet.methods.diverse import hinge_loss
from functools import partial

/home/birk/mambaforge-pypy3/envs/nbdev2/lib/python3.7/site-packages/haiku/_src/data_structures.py:37: FutureWarning: jax.tree_structure is deprecated, and will be removed in a future release. Use jax.tree_util.tree_structure instead.
  PyTreeDef = type(jax.tree_structure(None))


In [4]:
#| exporti
class Encoder(hk.Module):
    def __call__(self, x: jnp.DeviceArray, is_training: bool):
        mean = MLP([20, 16, 14, 12, 5], 0.3, name="EncoderMean")(x, is_training)
        var = MLP([20, 16, 14, 12, 5], 0.3, name="EncoderVar")(x, is_training)
        logvar = 0.5 + var
        return mean, logvar

class Decoder(hk.Module):
    def __init__(self, input_size):
        super().__init__()
        self.input_size: int = input_size

    def __call__(self, z: jnp.DeviceArray, is_training: bool):
        mean = MLP([12, 14, 16, 20, self.input_size], 0.3, name="DecoderMean")(z, is_training)
        return mean

In [5]:
#| export
class VariationalAutoEncoder(hk.Module):
    def __init__(self, input_size, name=None):
        super().__init__(name=name)
        self.encoder = Encoder()
        self.decoder = Decoder(input_size)
    # def encode(self, x: jnp.DeviceArray, is_training: bool = True):
    #     mean = MLP([20, 16, 14, 12, 5], 0.3, name="EncoderMean")(x, is_training)
    #     var = MLP([20, 16, 14, 12, 5], 0.3, name="EncoderVar")(x, is_training)
    #     logvar = 0.5 + var
    #     return mean, logvar
    
    # def decode(self, z: jnp.DeviceArray, input_size: int, is_training: bool = True):
    #     mean = MLP([12, 14, 16, 20, input_size], 0.3, name="DecoderMean")(z, is_training)
    #     return mean

    def sample_latent(self, mean, logvar):
        eps = jax.random.normal(hk.next_rng_key(), logvar.shape)
        return mean + jnp.sqrt(logvar) * eps

    def __call__(
        self, x: jnp.DeviceArray, y_target: jnp.DeviceArray, mc_samples: int = 50, is_training: bool = True
    ) -> jnp.DeviceArray:
        assert mc_samples >= 1, f"`mc_samples` should be >= 1 (got `mc_samples={mc_samples}`)"
        input_size = x.shape[-1]
        y_target = y_target.reshape(y_target.shape[0], 1)
        # TODO: vmap on Pytree
        em, ev = self.encoder(jnp.concatenate((x, y_target), 1), is_training)
        res = {
            'em': em, 'ev': ev, 'z': list(), 'x_pred': list(), 'mc_samples': mc_samples
        }
        for _ in range(mc_samples):
            z = self.sample_latent(em, ev)
            x_pred = self.decoder(jnp.concatenate((z, y_target), 1), is_training)
            res['z'].append(z); res['x_pred'].append(x_pred)
        return res

    # def compute_elbo(self, x, y_target):
    #     y_target = y_target.reshape(y_target.shape[0], 1)
    #     em, ev = self.encode(jnp.concatenate((x, y_target), 1))
    #     kl_divergence = 0.5 * jnp.mean(em**2 + ev - jnp.log(ev) - 1, axis=1)

    #     z = self.sample_latent(em, ev)
    #     dm = self.decode(jnp.concatenate((z, y_target), 1))
    #     log_px_z = jnp.array([0.])
    #     x_pred = dm
    #     return jnp.mean(log_px_z), jnp.mean(kl_divergence), x, x_pred



In [6]:
#| export
class VAECFConfigs(BaseParser):
    validity_reg: float
    lr: float

In [7]:
#| exporti
def make_vae() -> hk.Transformed:
    # example:
    # net = make_model(PredictiveModel)
    # params = net.init(...)
    def model_fn(x, y, mc_samples=1, is_training: bool = True):
        return VariationalAutoEncoder(x.shape[-1])(x, y, mc_samples, is_training)

    return hk.transform(model_fn)


def init_net_opt_vae(
    net: hk.Transformed,
    opt: optax.GradientTransformation,
    X: jnp.DeviceArray,
    y: jnp.DeviceArray,
    key: random.PRNGKey
) -> Tuple[hk.Params, optax.OptState]:
    X = device_put(X)
    params = net.init(key, X, y, is_training=True)
    opt_state = opt.init(params)
    return params, opt_state


In [47]:
#| export
class VAE_CF(BaseTrainingModule, BaseCFExplanationModule):
    name = "VAECF"

    def __init__(self, m_configs, pred_fn):
        self.save_hyperparameters(m_configs)
        self.configs = validate_configs(m_configs, VAECFConfigs)
        self.pred_fn = pred_fn
        self.net = make_vae()
        self.opt = optax.adam(learning_rate=self.configs.lr)

    def init_net_opt(self, dm, key):
        self.update_cat_info(dm)
        X = dm.get_sample_X()
        y = 1. - self.pred_fn(X)
        params, opt_state = init_net_opt_vae(
            self.net, self.opt, X=X, y=y, key=key)
        return params, opt_state

    @partial(jax.jit, static_argnames=['self', 'is_training'])
    def forward(self, params, rng_key, x, is_training: bool = True):
        y_target = 1. - self.pred_fn(x)
        y_target = jnp.round(y_target)
        res = self.net.apply(params, rng_key, x, y_target, mc_samples=1, is_training=is_training)
        cf = res['x_pred'][0]
        cf_y = self.pred_fn(cf)
        return cf, cf_y

    def predict(self, params, rng_key, x):
        return self.pred_fn(x)

    def generate_cfs(self, X: chex.ArrayBatched, params: Optional[hk.Params], rng_key: Optional[random.PRNGKey]) -> chex.ArrayBatched:
        cfs, _ = self.forward(params, rng_key, X, is_training=False)
        return cat_normalize(cfs, self.cat_arrays, self.cat_idx, hard=True)
    
    def compute_loss(self, out, x, y_target):
        em = out['em']
        ev = out['ev']
        z = out['z']
        dm = out['x_pred']
        mc_samples = out['mc_samples']

        # KL divergence
        kl_divergence = 0.5 * jnp.mean(em**2 + ev - jnp.log(ev) - 1, axis=1)
        
        validity_loss, recon_err = 0., 0.

        for i in range(mc_samples):
            x_pred = dm[i]
            # Proximity: L1 Loss
            recon_err += - jnp.sum(jnp.abs(x - x_pred), axis=1)

            # Sum to 1 over the categorical indexes of a feature
            for col in self.cat_arrays:
                cat_end_idx = self.cat_idx + len(col)
                temp = - \
                    jnp.abs(1.0 - jnp.sum(x_pred[:, self.cat_idx: cat_end_idx], axis=1))
                recon_err += temp

            # validity
            cf_y = self.pred_fn(x_pred)
            validity_loss += hinge_loss(cf_y, y_target)

        recon_err = recon_err / mc_samples
        validity_loss = - self.configs.validity_reg * validity_loss / mc_samples
        return - jnp.mean(recon_err - kl_divergence) - validity_loss

    def loss_fn(self, params, rng_key, x, y_target):
        out = self.net.apply(params, rng_key, x, y_target, is_training=True)
        loss = self.compute_loss(out, x, y_target)
        return loss

    @partial(jax.jit, static_argnames=['self'])
    def _training_step(self, params, opt_state, rng_key, batch):
        x, _ = batch
        y_target = 1. - self.pred_fn(x)

        grads = jax.grad(self.loss_fn)(params, rng_key, x, y_target)
        upt_params, opt_state = grad_update(grads, params, opt_state, self.opt)
        return upt_params, opt_state

    def training_step(self, params: hk.Params, opt_state: optax.OptState, rng_key: random.PRNGKey, batch: Tuple[jnp.array, jnp.array]) -> Tuple[hk.Params, optax.OptState]:
        return self._training_step(params, opt_state, rng_key, batch)

    def validation_step(self, params: hk.Params, rng_key: random.PRNGKey, batch: Tuple[jnp.array, jnp.array]) -> Dict[str, Any]:
        x, y = batch
        y_target = 1. - self.pred_fn(x)
        cf, cf_y = self.forward(params, rng_key, x, is_training=False)

        logs = {
            'val/validity': accuracy(y_target, cf_y),
            'val/proximity': proximity(x, cf)
        }
        self.log_dict(logs)
        return logs
        

Test

In [38]:
from cfnet.train import train_model
from cfnet.datasets import TabularDataModule
from cfnet.training_module import PredictiveTrainingModule
from cfnet.evaluate import generate_cf_results_cfnet, evaluate_cfs
from cfnet.utils import load_json
from copy import deepcopy

In [29]:
adult_configs = load_json('assets/configs/data_configs/credit_card.json')

In [30]:
dm = TabularDataModule(adult_configs['data_configs'])

In [31]:
pred_model = PredictiveTrainingModule(adult_configs['mlp_configs'])

In [32]:
t_configs = {
    'n_epochs': 10,
}


In [33]:
pred_params, _ = train_model(
    pred_model, dm, t_configs
)

/home/birk/mambaforge-pypy3/envs/nbdev2/lib/python3.7/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/home/birk/mambaforge-pypy3/envs/nbdev2/lib/python3.7/site-packages/haiku/_src/data_structures.py:145: FutureWarning: jax.tree_unflatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_unflatten instead.
  return jax.tree_unflatten(treedef, leaves)
/home/birk/code/cfnet/cfnet/_ckpt_manager.py:43: UserWarning: `monitor_metrics` is not specified in `CheckpointManager`. No checkpoints will be stored.
  warnings.warn("`monitor_metrics` is not specified in `CheckpointManager`. No checkpoints will be stored.")
Epoch 9: 100%|██████████| 88/88 [00:01<00:00, 75.31batch/s, train/train_loss_1=0.0836]


In [34]:
_params = deepcopy(pred_params)
pred_fn = lambda x: pred_model.forward(_params, random.PRNGKey(0), x, is_training=False)

In [48]:
vae_configs = {
    'validity_reg': 42.0, 'lr': 0.001
}
vae = VAE_CF(vae_configs, pred_fn)

In [49]:
params, _ = train_model(
    vae, dm, t_configs
)

Epoch 9: 100%|██████████| 88/88 [00:00<00:00, 137.47batch/s, val/proximity=7.5908494, val/validity=0.8614457]


In [50]:
cf_results = generate_cf_results_cfnet(vae, dm, params, random.PRNGKey(0))

In [51]:
evaluate_cfs(cf_results)

{('', 'VAECF'): {'acc': 0.8167999982833862,
  'validity': 0.8565333485603333,
  'proximity': 8.435470581054688}}